In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("data/suumo.csv")

In [3]:
df.isnull().sum()

name             3842
station             0
price            3842
sikikinreikin    3842
room             3842
age              3842
address          3842
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.head()

,name,station,price,sikikinreikin,room,age,address
0,アルテシモ サリーレ,東武伊勢崎線/曳舟駅 歩6分,7.3万円 管理費 15000円,敷- 礼7.3万円 保証金 - 敷引・償却 -,1K 25.25m2 東,マンション 築6年,東京都墨田区京島１
1,Forbitezza,東京メトロ半蔵門線/押上駅 歩8分,7.4万円 管理費 4000円,敷- 礼- 保証金 - 敷引・償却 -,1K 15.24m2 -,アパート 築8年,東京都墨田区押上２
2,京成押上線 押上駅 3階建 築8年,京成押上線/押上駅 歩4分,7.4万円 管理費 4000円,敷- 礼- 保証金 - 敷引・償却 -,1K 15.24m2 -,アパート 築8年,東京都墨田区押上２
3,東武伊勢崎線 押上駅 2階建 築6年,東武伊勢崎線/曳舟駅 歩8分,5.65万円 管理費 3500円,敷- 礼5.65万円 保証金 - 敷引・償却 -,ワンルーム 10.5m2 南東,アパート 築6年,東京都墨田区文花１
4,京成押上線 京成曳舟駅 5階建 築5年,京成押上線/押上駅 歩4分,9.7万円 管理費 10000円,敷9.7万円 礼9.7万円 保証金 - 敷引・償却 -,1LDK 30.09m2 -,マンション 築5年,東京都墨田区八広１


In [6]:
station_line_list = []
time_move_list = []

In [7]:
for i in df["station"]:
    station_line_list.append(i.split(" ")[0])
    time_move_list.append(i.split(" ")[1])

In [8]:
set(station_line_list)

{'京成押上線/押上駅',
 '東京メトロ半蔵門線/押上駅',
 '東武亀戸線/曳舟駅',
 '東武伊勢崎線/押上駅',
 '東武伊勢崎線/曳舟駅',
 '都営浅草線/押上駅'}

In [9]:
station_list = []
line_list = []

In [10]:
for i in station_line_list:
    station_list.append(i.split("/")[1])
    line_list.append(i.split("/")[0])

In [11]:
set(station_list)

{'押上駅', '曳舟駅'}

In [12]:
set(line_list)

{'京成押上線', '東京メトロ半蔵門線', '東武亀戸線', '東武伊勢崎線', '都営浅草線'}

In [13]:
bus_list = []
walk_list =[]
time_list = []

In [14]:
for i in time_move_list:
    if "歩" in i:
        bus_list.append(0)
        walk_list.append(1)
        time_list.append(re.sub(r"\D", "", i))
    else:
        bus_list.append(1)
        walk_list.append(0)
        time_list.append(re.sub(r"\D", "", i))

In [15]:
df["station"] = pd.Series(station_list)
df["line"] = pd.Series(line_list)
df["bus"] = pd.Series(bus_list)
df["walk"] = pd.Series(walk_list)
df["time"] = pd.Series(time_list).astype(float)

In [16]:
price_list = []
manage_list = []

In [17]:
for i in df["price"]:
    price_list.append(i.split(" ")[0].replace("万円",""))
    manage_list.append(i.split(" ")[3].replace("円",""))

In [18]:
manage_fee_list = []

In [19]:
for i in manage_list:
    if i == "-":
        manage_fee_list.append("0")
    else:
        manage_fee_list.append(i)

In [20]:
df["price"] = pd.Series(price_list).astype(float)
df["manage"] = pd.Series(manage_fee_list).astype(float)

In [21]:
sikikin_pre = []
reikin_pre = []
deposit_pre = []
sikibiki_pre = []

In [22]:
a = "敷9.75万円"

In [23]:
print(a.split("敷")[1])

9.75万円


In [24]:
for i in df["sikikinreikin"]:
    sikikin_pre.append(i.split(" ")[0])
    reikin_pre.append(i.split(" ")[2])
    deposit_pre.append(i.split(" ")[4])
    sikibiki_pre.append(i.split(" ")[6])

In [25]:
sikikin_list = []
reikin_list = []
deposit_list = []
sikibiki_list = []

In [26]:
for i in sikikin_pre:
    a = i.split("敷")[1]
    if "万円" in a:
        sikikin_list.append(a.replace("万円",""))
    else:
        sikikin_list.append(0)

for i in reikin_pre:
    a = i.split('礼')[1]
    if "万円" in a:
        reikin_list.append(a.replace("万円",""))
    else:
        reikin_list.append(0)

for i in deposit_pre:
    a = i.split('\xa0')[1]
    if "万円" in a:
        deposit_list.append(a.replace("万円",""))
    else:
        deposit_list.append(0)
        
for i in sikibiki_pre:
    a = i.split("\xa0")[1]
    if "万円" in a:
        sikibiki_list.append(a.replace("万円",""))
    elif "-" in a:
        sikibiki_list.append(0)
    else:
        sikibiki_list.append("実費")

In [27]:
df["sikikin"] = pd.Series(sikikin_list).astype(float)
df["reikin"] = pd.Series(reikin_list).astype(float)
df["deposit"] = pd.Series(deposit_list).astype(float)
df["sikibiki"] = pd.Series(sikibiki_list)

In [28]:
df["room"].unique()

array(['1K  25.25m2  東', '1K  15.24m2  -', 'ワンルーム  10.5m2  南東', ...,
       'ワンルーム  13.11m2  南', '1K  20.42m2  -', 'ワンルーム  18.13m2  南'],
      dtype=object)

In [29]:
room_type_pre = []
area_pre = []
direction_pre = []

In [30]:
for i in df["room"]:
    room_type_pre.append(i.split(" ")[0])
    area_pre.append(i.split(" ")[2])
    direction_pre.append(i.split(" ")[4])

In [31]:
room_list = []
L_list = []
D_list = []
S_list = []
K_list = []
R_list = []

In [32]:
for i in room_type_pre:
    if i == "ワンルーム":
        R_list.append(1)
    else:
        R_list.append(0)
        
    if "L" in i:
        L_list.append(1)
    else:
        L_list.append(0)
    
    if "D" in i:
        D_list.append(1)
    else:
        D_list.append(0)
        
    if "S" in i:
        S_list.append(1)
    else:
        S_list.append(0)
        
    if "K" in i:
        K_list.append(1)
    else:
        K_list.append(0)
    
    if i != "ワンルーム":
        room_list.append(re.sub(r"\D", "", i))
    else:
        room_list.append(0)

In [33]:
df["room"] = pd.Series(room_list).astype(int)
df["L"] = pd.Series(L_list).astype(int)
df["D"] = pd.Series(D_list).astype(int)
df["S"] = pd.Series(S_list).astype(int)
df["K"] = pd.Series(K_list).astype(int)
df["R"] = pd.Series(R_list).astype(int)

In [34]:
area_list = []

In [35]:
for i in area_pre:
    area_list.append(i.replace("m2",""))

In [36]:
df["area"] = pd.Series(area_list)

In [37]:
set(direction_pre)

{'-', '北', '北東', '北西', '南', '南東', '南西', '東', '西'}

In [38]:
direction_list = []

In [39]:
for i in direction_pre:
    if i == "-":
        direction_list.append("なし")
    else:
        direction_list.append(i)

In [40]:
df["direction"] = pd.Series(direction_list)

In [41]:
type_list = []
age_pre = []

In [42]:
a = "マンション  築6年"

In [43]:
a.split(" ")[2]

'築6年'

In [44]:
for i in df["age"]:
    type_list.append(i.split(" ")[0])
    age_pre.append(i.split(" ")[2])

In [45]:
age_list = []

In [46]:
for i in age_pre:
    if i == "新築":
        age_list.append(0)
    else:
        age_list.append(re.sub(r"\D", "", i))

In [47]:
df["type"] = pd.Series(type_list)
df["age"] = pd.Series(age_list).astype(float)

In [48]:
df = df.drop("sikikinreikin",axis=1)

In [49]:
df[df["sikibiki"]=="実費"]

,name,station,price,room,age,address,line,bus,walk,time,...,deposit,sikibiki,L,D,S,K,R,area,direction,type
7411,ＡＲ向島,押上駅,15.6,1,0.0,東京都墨田区向島５,都営浅草線,0,1,12.0,...,0.0,実費,1,1,0,1,0,41.01,西,マンション
7619,ＡＲ向島,曳舟駅,9.5,1,0.0,東京都墨田区向島５,東武亀戸線,0,1,11.0,...,0.0,実費,0,0,0,1,0,25.58,西,マンション


In [50]:
df["sikikin"][7411]

0.0

In [51]:
df["sikibiki"][7619]

'実費'

In [52]:
df["sikikin"][7619]

0.0

In [53]:
df["sikibiki"][7411] = 6.1

<ipython-input-53-9448d5c7b276>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sikibiki"][7411] = 6.1


In [54]:
df["sikibiki"][7619]  = 0.0

<ipython-input-54-e2bce3f1d924>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sikibiki"][7619]  = 0.0


In [55]:
df[df["sikibiki"]=="実費"]

,name,station,price,room,age,address,line,bus,walk,time,...,deposit,sikibiki,L,D,S,K,R,area,direction,type


In [56]:
df.columns

Index(['name', 'station', 'price', 'room', 'age', 'address', 'line', 'bus',
       'walk', 'time', 'manage', 'sikikin', 'reikin', 'deposit', 'sikibiki',
       'L', 'D', 'S', 'K', 'R', 'area', 'direction', 'type'],
      dtype='object')

In [57]:
df = df.reindex(columns=["name","price","age","manage","sikikin","reikin","deposit","sikibiki","line","station","bus","walk","time","room","R","L","D","S","K","area","direction","type","address"])

In [58]:
df

,name,price,age,manage,sikikin,reikin,deposit,sikibiki,line,station,...,room,R,L,D,S,K,area,direction,type,address
0,アルテシモ サリーレ,7.30,6.0,15000.0,0.0,7.30,0.0,0,東武伊勢崎線,曳舟駅,...,1,0,0,0,0,1,25.25,東,マンション,東京都墨田区京島１
1,Forbitezza,7.40,8.0,4000.0,0.0,0.00,0.0,0,東京メトロ半蔵門線,押上駅,...,1,0,0,0,0,1,15.24,なし,アパート,東京都墨田区押上２
2,京成押上線 押上駅 3階建 築8年,7.40,8.0,4000.0,0.0,0.00,0.0,0,京成押上線,押上駅,...,1,0,0,0,0,1,15.24,なし,アパート,東京都墨田区押上２
3,東武伊勢崎線 押上駅 2階建 築6年,5.65,6.0,3500.0,0.0,5.65,0.0,0,東武伊勢崎線,曳舟駅,...,0,1,0,0,0,0,10.5,南東,アパート,東京都墨田区文花１
4,京成押上線 京成曳舟駅 5階建 築5年,9.70,5.0,10000.0,9.7,9.70,0.0,0,京成押上線,押上駅,...,1,0,1,1,0,1,30.09,なし,マンション,東京都墨田区八広１
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551,ハイツ岡 203号室,7.00,41.0,0.0,7.0,0.00,0.0,7,東京メトロ半蔵門線,押上駅,...,2,0,0,0,0,1,33,南,マンション,東京都墨田区八広２
20552,東京メトロ半蔵門線 押上駅 4階建 築41年,7.00,41.0,0.0,7.0,0.00,0.0,7,東京メトロ半蔵門線,押上駅,...,2,0,0,0,0,1,33,南,マンション,東京都墨田区八広２
20553,ハイツ岡 203号室,7.00,41.0,0.0,7.0,0.00,0.0,7,東京メトロ半蔵門線,押上駅,...,2,0,0,0,0,1,33,南,マンション,東京都墨田区八広２
20554,ハイツ岡 203号室,7.00,41.0,0.0,7.0,0.00,0.0,7,東京メトロ半蔵門線,押上駅,...,2,0,0,0,0,1,33,南,マンション,東京都墨田区八広２


In [59]:
df.to_csv("data/suumo_pre.csv",index=False)